In [4]:
import numpy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils.data as Data
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

data=pd.read_excel('C:/Users/XHM/Desktop/data/TFT/trainlog.xlsx',engine='openpyxl')
all_input=data[['Vg','Vd','W/L']].values
all_output=data['Id1'].values
z=preprocessing.StandardScaler()
#all_output=minmax.fit_transform(data['Idrain'].values.reshape(-1,1))

(x_train,x_test,y_train,y_test)=train_test_split(all_input,all_output,train_size=0.8,
                                                 random_state=0)
a=np.mean(x_train, axis = 0)
print(a)
#标准化处理
x_train=z.fit_transform(x_train)
x_test=z.transform(x_test)
#z.inverse_transform



#数据类型处理
train_xt=torch.from_numpy(x_train.astype(np.float32))
train_yt=torch.from_numpy(y_train.astype(np.float32))
test_xt=torch.from_numpy(x_test.astype(np.float32))
test_yt=torch.from_numpy(y_test.astype(np.float32))
#将数据处理为数据加载器
train_data=Data.TensorDataset(train_xt,train_yt.float())
test_data=Data.TensorDataset(test_xt,test_yt.float())

train_loader=Data.DataLoader(dataset=train_data,batch_size=32,shuffle=True,num_workers=0)
test_loader=Data.DataLoader(dataset=test_data,batch_size=32,shuffle=False,num_workers=0)


[10.48288651  8.35886953  1.66376229]


In [5]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,make_scorer
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor

# 对MLP应用贝叶斯

In [12]:
def rf_cv(hid_1, hid_2):  #优化目标
    mlp=MLPRegressor(hidden_layer_sizes=(int(hid_1),int(hid_2)),
        max_iter=1000,
        activation='tanh',
        solver='adam',
        learning_rate_init=0.001,
        shuffle =True,
        batch_size=32)
    mlp.fit(train_xt,train_yt)
    pre=mlp.predict(test_xt)    
    val=r2_score(test_yt,pre)
    #val=mlp.score(train_xt,train_yt)
    return val
rf_bo = BayesianOptimization(
        rf_cv,
        {'hid_1': (5, 15),
        'hid_2': (5, 15),
        }
    )
rf_bo.maximize()

|   iter    |  target   |   hid_1   |   hid_2   |
-------------------------------------------------
|  1        |  0.9946   |  14.49    |  7.648    |
|  2        |  0.9948   |  11.6     |  10.07    |
|  3        |  0.9955   |  9.213    |  12.22    |
|  4        |  0.9949   |  14.27    |  9.089    |
|  5        |  0.9941   |  6.241    |  12.52    |
|  6        |  0.9948   |  14.58    |  11.37    |
|  7        |  0.9942   |  14.94    |  8.977    |
|  8        |  0.9942   |  14.6     |  11.38    |
|  9        |  0.9946   |  7.411    |  7.807    |
|  10       |  0.9954   |  11.55    |  9.366    |
|  11       |  0.9948   |  13.7     |  12.24    |
|  12       |  0.994    |  11.76    |  12.99    |
|  13       |  0.9936   |  9.936    |  8.938    |
|  14       |  0.9941   |  13.27    |  9.15     |
|  15       |  0.9952   |  12.49    |  14.46    |
|  16       |  0.9944   |  6.85     |  14.43    |
|  17       |  0.9929   |  8.777    |  7.28     |
|  18       |  0.9941   |  6.25     |  9.461    |


# 对随机森林应用贝叶斯

In [6]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    rfr=RandomForestRegressor(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2)
    rfr.fit(train_xt,train_yt)
    pre=rfr.predict(train_xt)    
    val=r2_score(train_yt,pre)
    #val=rfr.score(train_xt,train_yt)
    return val
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )
rf_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9991   |  12.1     |  0.15     |  4.678    |  34.56    |
|  2        |  0.9972   |  8.083    |  0.2582   |  2.41     |  158.4    |
|  3        |  0.9996   |  9.8      |  0.9889   |  15.37    |  63.86    |
|  4        |  0.9998   |  11.88    |  0.9781   |  11.76    |  130.3    |
|  5        |  0.9992   |  12.32    |  0.8277   |  21.1     |  113.5    |
|  6        |  0.9994   |  14.31    |  0.2043   |  7.486    |  121.7    |
|  7        |  0.9908   |  5.363    |  0.9233   |  24.36    |  133.4    |
|  8        |  0.999    |  12.04    |  0.1215   |  11.02    |  129.0    |
|  9        |  0.999    |  14.65    |  0.435    |  9.22     |  136.3    |
|  10       |  0.9999   |  12.23    |  0.9564   |  10.73    |  112.6    |
|  11       |  0.9999   |  14.79    |  0.3409   |  2.286    |  113.4    |
|  12       |  0.9908   |  5.0      | 

In [ ]:
import GPy
import GPyOpt

from GPyOpt.methods import BayesianOptimization
bounds = np.array([[0, 50]])

def rf_cv(hid_1, hid_2):  #优化目标
    mlp=MLPRegressor(hidden_layer_sizes=(int(hid_1),int(hid_2)),
        max_iter=1000,
        activation='logistic',
        solver='adam',
        learning_rate_init=0.004,
        shuffle =True,
        batch_size=64)
    mlp.fit(train_xt,train_yt)
    pre=mlp.predict(test_xt)    
    val=r2_score(test_yt,pre)
    #val=mlp.score(train_xt,train_yt)
    return val
kernel = GPy.kern.Matern52(input_dim=2, variance=1.0, lengthscale=1.0)

params={'n_hidden_l1': (32, 256),
        'n_hidden_l2': (32, 256),}
bds=params

optimizer = BayesianOptimization(f=rf_cv, 
                                 domain=bds,
                                
                                 model_type='GP',
                                 kernel=kernel,
                                 acquisition_type ='EI',
                                 acquisition_jitter = 0.01,
                                 exact_feval=False,
                                 normalize_Y=False,
                                 maximize=True)

optimizer.run_optimization(max_iter=10)
optimizer.plot_acquisition()
optimizer.plot_convergence()